In [1]:
import anndata
import numpy as np
import sys
import torch
sys.path.append('../../../../')
import velovae as vv

In [2]:
import scvelo as scv
dataset = 'Dentategyrus'
root = '/scratch/blaauw_root/blaauw1/gyichen'
adata = anndata.read_h5ad(f'{root}/data/{dataset}_pp.h5ad')

In [ ]:
# vv.preprocess(adata, n_gene=2000, min_shared_counts=20)

In [3]:
model_path_base = f'{root}/checkpoints/{dataset}'
figure_path_base = f'{root}/figures/{dataset}'
data_path = f'{root}/data/velovae/discrete/{dataset}'
gene_plot = ['Tmsb10', 'Fam155a', 'Hn1', 'Rpl6']

# VeloVAE

In [10]:
figure_path = f'{figure_path_base}/DVAE'
model_path = f'{model_path_base}/DVAE'

torch.manual_seed(2022)
np.random.seed(2022)
vae = vv.VAE(adata, 
             tmax=20, 
             dim_z=5, 
             device='cuda:0',
             discrete=True)

vae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

vae.save_model(model_path, 'encoder', 'decoder')
vae.save_anndata(adata, 'dvae', data_path, file_name=f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Estimating ODE parameters...


  0%|          | 0/1000 [00:00<?, ?it/s]

Detected 255 velocity genes.
Estimating the variance...


  0%|          | 0/1000 [00:00<?, ?it/s]

Initialization using the steady-state and dynamical models.
Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/1000 [00:00<?, ?it/s]

3 clusters detected based on gene co-expression.
(0.53, 0.7940861417910453), (0.47, 0.2627669307941492)
(0.49, 0.7775889478200321), (0.51, 0.2601876073663219)
(0.30, 0.3421788138328361), (0.70, 0.8284624210566714)
KS-test result: [0. 0. 0.]
Initial induction: 602, repression: 398/1000
Learning Rate based on Data Sparsity: 0.0004
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 17, test iteration: 32
*********                      Stage  2                       *********
*********             Velocity R

  0%|          | 0/2930 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.030
Average Set Size: 59
*********     Round 1: Early Stop Triggered at epoch 1252.    *********
*********             Velocity Refinement Round 2             *********
Change in x0: 0.2763
*********             Velocity Refinement Round 3             *********
*********     Round 3: Early Stop Triggered at epoch 1968.    *********
Change in x0: 0.1523
*********             Velocity Refinement Round 4             *********
Change in x0: 0.0925
*********             Velocity Refinement Round 5             *********
*********     Round 5: Early Stop Triggered at epoch 2541.    *********
Change in x0: 0.0884
*********             Velocity Refinement Round 6             *********
Stage 2: Early Stop Triggered at round 5.
*********              Finished. Total Time =   0 h : 10 m :  4 s             *********
Final: Train ELBO = -827.789,	Test ELBO = -845.458


# Full VB

In [11]:
figure_path = f'{figure_path_base}/DFullVB'
model_path = f'{model_path_base}/DFullVB'

torch.manual_seed(2022)
np.random.seed(2022)
full_vb = vv.VAE(adata, 
                 tmax=20, 
                 dim_z=5, 
                 device='cuda:0',
                 discrete=True,
                 full_vb=True)

full_vb.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

full_vb.save_model(model_path, 'encoder', 'decoder')
full_vb.save_anndata(adata, 'dfullvb', data_path, file_name=f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Estimating ODE parameters...


  0%|          | 0/1000 [00:00<?, ?it/s]

Detected 255 velocity genes.
Estimating the variance...


  0%|          | 0/1000 [00:00<?, ?it/s]

Initialization using the steady-state and dynamical models.
Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/1000 [00:00<?, ?it/s]

3 clusters detected based on gene co-expression.
(0.53, 0.7940861417910453), (0.47, 0.2627669307941492)
(0.49, 0.7775889478200321), (0.51, 0.2601876073663219)
(0.30, 0.3421788138328361), (0.70, 0.8284624210566714)
KS-test result: [0. 0. 0.]
Initial induction: 602, repression: 398/1000
Learning Rate based on Data Sparsity: 0.0004
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 17, test iteration: 32
*********       Stage 1: Early Stop Triggered at epoch 816.       *********
*********                   

  0%|          | 0/2930 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.030
Average Set Size: 60
*********     Round 1: Early Stop Triggered at epoch 939.    *********
*********             Velocity Refinement Round 2             *********
*********     Round 2: Early Stop Triggered at epoch 1107.    *********
Change in x0: 0.3256
*********             Velocity Refinement Round 3             *********
*********     Round 3: Early Stop Triggered at epoch 1149.    *********
Change in x0: 0.2299
*********             Velocity Refinement Round 4             *********
*********     Round 4: Early Stop Triggered at epoch 1496.    *********
Change in x0: 0.1440
*********             Velocity Refinement Round 5             *********
*********     Round 5: Early Stop Triggered at epoch 1640.    *********
Change in x0: 0.1321
*********             Velocity Refinement Round 6             *********
*********     Round 6: Early Stop Triggered at epoch 1667.    *********
Change in x0: 0.1116
*********             Velocity Refinement Round 7

# Evaluation

In [12]:
vv.post_analysis(adata,
                 'eval',
                 ['Discrete VeloVAE','Discrete FullVB'],
                 ['dvae','dfullvb'],
                 compute_metrics=False,
                 raw_count=True,
                 genes=gene_plot,
                 grid_size=(1,4),
                 figure_path=figure_path_base)

---   Plotting  Results   ---
saving figure to file /scratch/blaauw_root/blaauw1/gyichen/figures/Dentategyrus/eval_dvae_stream.png
saving figure to file /scratch/blaauw_root/blaauw1/gyichen/figures/Dentategyrus/eval_dfullvb_stream.png


(None, None)